In [39]:
import numpy as np
import os
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding,Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
import sys
import re
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)


In [15]:
path=[x for x in os.scandir(r"C:\Users\trilo\Desktop\Proyectos\Periodista\Noticias")]

In [16]:
text = "".join([open(x).read().lower() for x in path])
len(text)

127916

In [95]:
res=lambda x: re.sub('[^A-Za-zá-ú\s]','',x)
splitten=res(text).lower()
maxlen = 40
step =1
sentences = []
next_chars = []

chars = sorted(list(set(splitten)))
char_indices=dict((char, chars.index(char)) for char in chars)

for i in range(0,len(splitten)-maxlen,step):
    sentences.append(np.array([char_indices[x] for x in splitten[i:i+maxlen]]))
    next_chars.append(char_indices[splitten[i+maxlen]])
print('Secuencias:', len(sentences))
#------------------------------------->>>>termina generacion de secuencias


Secuencias: 124150


In [96]:
indices_char=dict([(x[1],x[0]) for x in char_indices.items()])

In [97]:
def calculate(sentencia):
    sentencia=np.array(sentencia)
    x=np.zeros((len(sentencia),maxlen,len(chars)))
    for i in range(len(sentencia)):
        for k in range(maxlen):
            x[i,k,sentencia[i,k]]=1
    return x

In [98]:
x=calculate(sentences)
y=np.array(next_chars)

In [134]:
adam=Adam(learning_rate=.001)

In [100]:
L=len(chars)

In [101]:
entrada0=Input((x.shape[1:]))
capa_10=Bidirectional(LSTM(100,return_sequences=False,return_state=False,))(entrada0)
# capa_10=Bidirectional(LSTM(100,return_sequences=False,return_state=False))(capa_10)
salida0=Dense(L,activation="softmax")(capa_10)
Gen=Model(entrada0,salida0)
Gen.compile(loss="sparse_categorical_crossentropy",optimizer=adam,metrics=['acc'])

In [102]:
Gen.summary()

Model: "functional_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 40, 34)]          0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 200)               108000    
_________________________________________________________________
dense_10 (Dense)             (None, 34)                6834      
Total params: 114,834
Trainable params: 114,834
Non-trainable params: 0
_________________________________________________________________


In [103]:
xtr,xts,ytr,yts=train_test_split(x,y,test_size=.1,random_state=88)

In [142]:
history=Gen.fit(xtr, ytr,batch_size=105,epochs=5,validation_split=.1)

Epoch 1/5
958/958 [==============================] - 51s 53ms/step - loss: 0.8803 - acc: 0.7167 - val_loss: 1.6854 - val_acc: 0.5707
Epoch 2/5
958/958 [==============================] - 50s 52ms/step - loss: 0.8781 - acc: 0.7172 - val_loss: 1.6573 - val_acc: 0.5686
Epoch 3/5
958/958 [==============================] - 51s 53ms/step - loss: 0.8750 - acc: 0.7180 - val_loss: 1.7192 - val_acc: 0.5576
Epoch 4/5
958/958 [==============================] - 50s 52ms/step - loss: 0.8828 - acc: 0.7162 - val_loss: 1.6855 - val_acc: 0.5711
Epoch 5/5
958/958 [==============================] - 51s 54ms/step - loss: 0.8826 - acc: 0.7148 - val_loss: 1.7059 - val_acc: 0.5654


In [144]:
Gen.save_weights(r'C:\Users\trilo\text_generator\text_generator_2.h5')

In [74]:
Gen.load_weights(r'C:\Users\trilo\text_generator\text_generator.h5')

In [106]:
def predecir(texto,total_len=100):
    texto=texto.lower()
    sentencia=[char_indices[x] for x in texto]
    predicciones=[]
    if len(sentencia)== maxlen:
        for i in range(total_len):
            x=calculate([sentencia[i:]])
            pred=np.argmax(Gen.predict(x)[0])
            predicciones.append(pred)
            sentencia.append(pred)
    else:
        print("faltan {} palabras para completar el input".format(str(maxlen-len(sentencia))))
    return ''.join([indices_char[x] for x in sentencia])

In [143]:
pred=predecir("la vacuna contra el coronavirus ya llego",total_len=750)
print(pred)

la vacuna contra el coronavirus ya llego asegura de la casa rosada y la solicitada al proyecto de la cámara de casación el gobierno de la vacuna de un problo a la postura de la propieña de la provincia de manera al gobierno en la cámara federal de la militancia de que su condentima la constitucional de la cámara federal a la corte que está la semana de la sesión de la postura vez que se esperabla de la constitucionar porque toca en el proyecto de impuesto y saber la casa rosada de la vicepresidenta a presupuesto en la corte de será de la sesión por el medior son todos con la masitación de la vacuna sino con el proyecto de la vacuna confiaron a la corte gonenciar a representante a la semana de conocimiento de la corte se había constitucional de la próxima de la renta de la mayoría


In [139]:
print(predecir("el presidente indico el fin de la cuare ".lower(),total_len=750))

el presidente indico el fin de la cuare de habla de la defensa na acuerdo a la cantidad de la corte para de mauricio macri es el congreso de la postfilma de la causa de este momento de la comunidad de los traslados de la corte segunda de la cercante que la comitiva es lo que la capitada a la comisión de los traslados de que el casa de más conscirnas de aportantes de la construción de los gobernadores y de la casa rosada de los procuradores y el caso


el caso de la cuestión de cara respondió
el presidente del consejo de juntos por el cambio en la corte suprema con los camaristas de la mayoría su consegión y del resto de la comisión de la por hablar justicia que la comitiva de impulsamos por el traslados con el kirchnerista de conochos y el deparel de buenos aires de la ciudad de 


In [141]:
print(predecir("la corte suprema aprobo el fallo final  ".lower(),total_len=750))

la corte suprema aprobo el fallo final  millones de la corte entre los traslados en el consejo para el caso de la mayoría que tiene este fueron los desatorisses en la conduta el presidente de diputados que en la corte hubo y el debate con la corte segurió que el proyecto


en la causa a la mitad de los provincias que con la cámara de casación del gobierno por la vacuna de la provincia de propio de la propia para sin el cambio de los gobernadores del gobierno por el caso el directores de la corte patra de la presenta en la corte suprema de la corte suprema de mauricio macrice con los recursos de la cámara de casacias con el fmitambre de vacunamiento propia confirmó a la aplicación para tiene que esta límita en el caso diferentes de conocimiento y al procedimbre y la propuesta en e
